![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2F04+-+scikit-learn&dt=04f+-+Vertex+AI+Custom+Model+-+scikit-learn+-+Training+Pipeline+With+Custom+Container.ipynb)

# 04f - Vertex AI > Training > Training Pipelines - With Custom Container

**04 Series Overview**

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `04a-04i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  Custom Job that trains and saves (to GCS) a model from a python script (`04a`), python source distribution (`04b`), and custom container (`04c`)
-  Training Pipeline that trains and registers a model from a python script (`04d`), python source distribution (`04e`), and custom container (`04f`)
-  Hyperparameter Tuning Jobs from a python script (`04g`), python source distribution (`04h`), and custom container (`04i`)

**This Notebook (`04f`): An extension of `04c` as a Training Pipeline that saves the model to Vertex AI > Model Registry**

This notebook trains the same scikit-learn logistic regression model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb). 

A custom container is built that contains the script and required Python libaries.  For more guidance on this process visit the tip notebook [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb).  The script, along with a `requirement.txt` and `Dockerfile` are stored in GCS and then used by Cloud Build to extend the pre-built Vertex AI container and store the resulting image in Artifact Registry.  

Compared to the Custom Job in `04c` the primary considerations for this Training Pipeline are:
- the final model is registered to Vertex AI > Model Registry
- the training pipeline triggers a similar custom job in Vertex AI > Training

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - [`aiplatform.CustomContainerTrainingJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomContainerTrainingJob#google_cloud_aiplatform_CustomContainerTrainingJob)
            - and run with the method [`.run()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomContainerTrainingJob#google_cloud_aiplatform_CustomContainerTrainingJob_run)

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
- [Vertex AI Custom Container For Training](https://cloud.google.com/vertex-ai/docs/training/containers-overview)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05f_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05f_console.png" width="45%">
</p>

---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry and Cloud Build)
- Artifact Registry
    - to create repositories for Python packages and Docker containers
- Cloud Build
    - To build custom Docker containers

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [1]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [2]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    !pip install google-cloud-artifact-registry -q

In [3]:
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print('You need to pip install google-cloud-build')
    !pip install google-cloud-build

### Environment

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [5]:
REGION = 'us-central1'
EXPERIMENT = '04f'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/sklearn-cpu.0-23'
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id-splits' + '-' + VAR_TARGET # add more variables to the string with space delimiters
SOLVER = 'newton-cg'
PENALTY = 'l2'

packages:

In [6]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

clients:

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()
gcs = storage.Client()
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

parameters:

In [8]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"
BLOB = f"{SERIES}/{EXPERIMENT}/models/{TIMESTAMP}/model/model.pkl"

In [9]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

List the service accounts current roles:

In [10]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.admin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [11]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [12]:
FRAMEWORK = 'sklearn'
TASK = 'classification'
MODEL_TYPE = 'logistric-regression'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

### Enable APIs

Using Cloud Build and Artifact Registry requires enabling these APIs for the Google Cloud Project.

Options for enabeling these.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Artifact Registry

In [13]:
artifactregistry = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
    )
).state.name


if artifactregistry == 'DISABLED':
    print(f'Artifact Registry is currently {artifactregistry} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Artifact Registry is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Artifact Registry already enabled for project: {PROJECT_ID}')

Artifact Registry already enabled for project: mg-ce-demos


#### Cloud Build

In [14]:
cloudbuild = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
    )
).state.name


if cloudbuild == 'DISABLED':
    print(f'Cloud Build is currently {cloudbuild} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Build is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Build already enabled for project: {PROJECT_ID}')

Cloud Build already enabled for project: mg-ce-demos


---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.scikit-learn.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [15]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [16]:
tb.resource_name

'projects/633472233130/locations/us-central1/tensorboards/4577495604850065408'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [17]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Python File for Training

This notebook trains the same scikit-learn Keras model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb) which stores the script in [`./code/train.py`](./code/train.py).

**Review the script:**

In [18]:
SCRIPT_PATH = './code/train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pickle
import pandas as pd 
import numpy as np 

from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import storage
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--penalty', dest = 'penalty', default = 'l2', type = str, help = 'Penalty term')
parser.add_argument('--solver', dest = 'solver', default = 'newton-cg', type = str, help = 'Logistic regression solver')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
parser.add_argument('--bucket', dest = 'bucket', type=str)
parser.add_argument('--uri', dest = 'uri', type=str)
parser.add_argument('--blob', dest = 'blob', type=str)
parser.add_argument('--timestamp', dest = 'timestamp', type=str)
args = parser.parse_args()

# Model Training
VAR_TARGET = str(args.var_target)
VAR_OMIT = str(args.var_omit).split('-')

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)

# Vertex AI Experiment
expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})

# get schema from bigquery source
query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': VAR_TARGET})

train_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TRAIN'"
train = bq.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[VAR_TARGET].astype('int')

val_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'VALIDATE'"
val = bq.query(val_query).to_dataframe()
X_val = val.loc[:, ~val.columns.isin(VAR_OMIT)]
y_val = val[VAR_TARGET].astype('int')

test_query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT)]
y_test = test[VAR_TARGET].astype('int')

# Logistic Regression
# instantiate the model 
logistic = LogisticRegression(solver=args.solver, penalty=args.penalty)

# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

expRun.log_params({'solver': args.solver, 'penalty': args.penalty})

# define pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)])

# define grid search model
model = pipe.fit(X_train, y_train)

# test evaluations:
y_pred = model.predict(X_test)
test_acc = metrics.accuracy_score(y_test, y_pred) 
test_prec = metrics.precision_score(y_test, y_pred)
test_rec = metrics.recall_score(y_test, y_pred)
test_rocauc = metrics.roc_auc_score(y_test, y_pred)
expRun.log_metrics({'test_accuracy': test_acc, 'test_precision': test_prec, 'test_recall': test_rec, 'test_roc_auc': test_rocauc})

# val evaluations:
y_pred_val = model.predict(X_val)
val_acc = metrics.accuracy_score(y_val, y_pred_val) 
val_prec = metrics.precision_score(y_val, y_pred_val)
val_rec = metrics.recall_score(y_val, y_pred_val)
val_rocauc = metrics.roc_auc_score(y_val, y_pred_val)
expRun.log_metrics({'validation_accuracy': val_acc, 'validation_precision': val_prec, 'validation_recall': val_rec, 'validation_roc_auc': val_rocauc})

# training evaluations:
y_pred_training = model.predict(X_train)
training_acc = metrics.accuracy_score(y_train, y_pred_training) 
training_prec = metrics.precision_score(y_train, y_pred_training)
training_rec = metrics.recall_score(y_train, y_pred_training)
training_rocauc = metrics.roc_auc_score(y_train, y_pred_training)
expRun.log_metrics({'training_accuracy': training_acc, 'training_precision':training_prec, 'training_recall': training_rec, 'training_roc_auc': training_rocauc})

# output the model save files
with open('model.pkl','wb') as f:
    pickle.dump(model,f)
    
# Upload the model to GCS
bucket = storage.Client().bucket(args.bucket)
blob = bucket.blob(args.blob)
blob.upload_from_filename('model.pkl')

expRun.log_params({'model.save': f'{args.uri}/models/{args.timestamp}/model'})
expRun.end_run()

```

### Creating a Custom Container with Cloud Build

Cloud Build creates and manages the build on GCP.  The API creates a build by providing:
- location of the source
- instructions
- location to store the built artifacts

The instruction part of Cloud Build has options:
- Dockerfile
- Build Config file (YAML or JSON)
- Cloud Native Buildpacks

This notebook uses the approach of using the Python Client for Cloud Build and not referencing any local files.  For that reason, the first step is creating a Dockerfile for the workflow and storing it in GCS. The next step is running Cloud Build and using the client to specify the Build config rather than a config file.  The steps of the build config start with getting the code (git clone, or copy from GCS) and copying the Dockerfile.  

There are many workflows for creating containers with ML training code.  Many of the most common ones are explored in the tips notebook [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb).  The method used here is the simplest - copy the training code directly into the container.  The other methods include packaging the training code as a Python Distribution and using `pip install` in from GCS, GitHub and even Artifact Registry as a private repository.

#### Store Resources in Cloud Storage

In [19]:
bucket = gcs.lookup_bucket(PROJECT_ID)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/training'

#### Copy Training Code

In [20]:
blob = bucket.blob(f'{SOURCEPATH}/{EXPERIMENT}_trainer/train.py')
blob.upload_from_filename(SCRIPT_PATH)

#### Create Requirements.txt File for Python

In [21]:
requirements = f"""google-cloud-aiplatform
protobuf
db-dtypes>=1.0.0
google-auth>=2.6.0
google-cloud-bigquery>=3.0.1
"""

In [22]:
blob = bucket.blob(f'{SOURCEPATH}/requirements.txt')
blob.upload_from_string(requirements)

#### Create the Dockerfile
A basic dockerfile thats take the base image and copies the code in and define an entrypoint - what python script to run first in this case.  Add RUN entries to pip install additional packages.

In [23]:
dockerfile = f"""
FROM {BASE_IMAGE}
WORKDIR /training
# copy requirements and install them
COPY requirements.txt ./
RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt
## Copies the trainer code to the docker image
COPY {EXPERIMENT}_trainer/* ./{EXPERIMENT}_trainer/
## Sets up the entry point to invoke the trainer
ENTRYPOINT ["python", "-m", "{EXPERIMENT}_trainer.train"]
"""

In [24]:
blob = bucket.blob(f'{SOURCEPATH}/Dockerfile')
blob.upload_from_string(dockerfile)

#### Setup Artifact Registry

Artifact registry organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

##### List Repositories

This may be empty if no repositories have been created for this project

In [25]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/mg-ce-demos/locations/us-central1/repositories/mg-ce-demos


#### Create Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create!

In [26]:
docker_repo = None
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{PROJECT_ID}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar_client.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{PROJECT_ID}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {EXPERIMENT} experiment that holds docker images.',
                name = f'{PROJECT_ID}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES, 'experiment': EXPERIMENT}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Retrieved existing repo: projects/mg-ce-demos/locations/us-central1/repositories/mg-ce-demos


In [27]:
docker_repo.name, docker_repo.format_.name

('projects/mg-ce-demos/locations/us-central1/repositories/mg-ce-demos',
 'DOCKER')

In [28]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

#### Build Custom Container
Use the Cloud Build client to construct and run the build instructions.  Here the files collected in GCS are copied to the build instance, then the Docker build in run in the folder with the `Dockerfile`.  The resulting image is pushed to Artifact Registry (setup above).

In [29]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}_trainer', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}_trainer"]

In [30]:
build

steps {
  name: "gcr.io/cloud-builders/gsutil"
  args: "cp"
  args: "-r"
  args: "gs://mg-ce-demos/04/04f/training/*"
  args: "/workspace"
}
steps {
  name: "gcr.io/cloud-builders/docker"
  args: "build"
  args: "-t"
  args: "us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/04f_trainer"
  args: "/workspace"
}
images: "us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/04f_trainer"

In [31]:
operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [32]:
response = operation.result()
response.status, response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/mg-ce-demos/mg-ce-demos/04f_trainer")

In [33]:
print(f"Review the Custom Container with Artifact Registry in the Google Cloud Console:\nhttps://console.cloud.google.com/artifacts/docker/{PROJECT_ID}/{REGION}/{PROJECT_ID}-docker?project={PROJECT_ID}")

Review the Custom Container with Artifact Registry in the Google Cloud Console:
https://console.cloud.google.com/artifacts/docker/mg-ce-demos/us-central1/mg-ce-demos-docker?project=mg-ce-demos


### Setup Training Job

In [34]:
CMDARGS = [
    "--penalty=" + PENALTY,
    "--solver=" + SOLVER,
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME,
    "--bucket=" + BUCKET,
    "--uri=" + URI,
    "--blob=" + BLOB,
    "--timestamp=" + TIMESTAMP
]

In [35]:
trainingJob = aiplatform.CustomContainerTrainingJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    container_uri = f"{REPOSITORY}/{EXPERIMENT}_trainer",
    model_serving_container_image_uri = DEPLOY_IMAGE,
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job AND Upload The Model
The training job will automatically upload the model to the Vertex AI Model Registry and return the link to the model.

In [36]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name
else:
    print('This is a new model, creating in model registry')
    parent_model = ''
    
if upload_model:
    model = trainingJob.run(
        model_display_name = f'{SERIES}_{EXPERIMENT}',
        model_labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'},
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model = parent_model,
        is_default_version = True,
        model_version_aliases = [RUN_NAME],
        model_version_description = RUN_NAME,
        base_output_dir = f"{URI}/models/{TIMESTAMP}",
        service_account = SERVICE_ACCOUNT,
        args = CMDARGS,
        replica_count = 1,
        machine_type = TRAIN_COMPUTE,
        accelerator_count = 0,
        tensorboard = tb.resource_name
    )

This is a new model, creating in model registry
Training Output directory:
gs://mg-ce-demos/04/04f/models/20230201100605 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2178597765481758720?project=633472233130
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7946099203120300032?project=633472233130
View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+633472233130+locations+us-central1+tensorboards+4577495604850065408+experiments+7946099203120300032
CustomContainerTrainingJob projects/633472233130/locations/us-central1/trainingPipelines/2178597765481758720 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/633472233130/locations/us-central1/trainingPipelines/2178597765481758720 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/633472233130/locations/us-central1/trainingPipelines/2

Get the backing Custom Job for the Training Pipeline:

In [37]:
clientPL = aiplatform.gapic.PipelineServiceClient(client_options = {'api_endpoint': f'{REGION}-aiplatform.googleapis.com'})

In [38]:
from google.protobuf.json_format import MessageToDict

backingCustomJob = MessageToDict(clientPL.get_training_pipeline(name = trainingJob.resource_name)._pb)['trainingTaskMetadata']['backingCustomJob']

In [39]:
customJob = aiplatform.CustomJob.get(backingCustomJob)
customJob.resource_name, customJob.display_name

('projects/633472233130/locations/us-central1/customJobs/7946099203120300032',
 '04_04f_20230201100605-custom-job')

Create hyperlinks to job and tensorboard here:

In [40]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
#board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.name.split('/')[-1]}"

print(f'Review the Training Pipeline here:\nhttps://console.cloud.google.com/vertex-ai/training/training-pipelines?project={PROJECT_ID}')
print(f'Review the Custom Job here:\n{job_link}')
#print(f'Review the TensorBoard From the Job here:\n{board_link}')
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

Review the Training Pipeline here:
https://console.cloud.google.com/vertex-ai/training/training-pipelines?project=mg-ce-demos
Review the Custom Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/7946099203120300032/cpu?cloudshell=false&project=mg-ce-demos
Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_04_04f?project=mg-ce-demos


---
## Serving

### Vertex AI Experiment Update and Review

In [41]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [42]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'trainingPipelines.display_name': trainingJob.display_name,
    'trainingPipelines.resource_name': trainingJob.resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    #'customJobs.tensorboard': board_link
})

Complete the experiment run:

In [43]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [44]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [45]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.var_target,param.penalty,param.customJobs.link,param.model.version_id,param.trainingPipelines.display_name,param.data_source,...,metric.test_accuracy,metric.validation_accuracy,metric.validation_precision,metric.test_roc_auc,metric.test_precision,metric.training_accuracy,metric.training_recall,metric.training_precision,metric.validation_roc_auc,metric.test_recall
0,experiment-04-04f-sklearn-classification-logis...,run-20230201100605,system.ExperimentRun,COMPLETE,Class,l2,https://console.cloud.google.com/vertex-ai/loc...,1,04_04f_20230201100605,bq://mg-ce-demos.fraud.fraud_prepped,...,0.999087,0.999016,0.848485,0.803501,0.894737,0.99921,0.618182,0.878229,0.774422,0.607143


Review the Experiments TensorBoard to compare runs:

In [46]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+633472233130+locations+us-central1+tensorboards+4577495604850065408+experiments+experiment-04-04f-sklearn-classification-logistric-regression


In [47]:
expRun.get_time_series_data_frame()

""


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [48]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [49]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [50]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-04-04f-sklearn-classification-logistric-regression
experiment-04-04e-sklearn-classification-logistric-regression
experiment-04-04d-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-logistic-regression
experiment-04-04c-sklearn-classification-logistric-regression
experiment-04-04b-sklearn-classification-logistric-regression
experiment-04-04a-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-sklearn-logistic-regression
experiment-04-04-tf-classification-sklearn-logistic-regression


Create ranks for models within experiment and across the entire SERIES:

In [52]:
def ranker(metric = 'metric.test_roc_auc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_roc_auc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
30,experiment-04-04-sklearn-classification-logist...,run-20221102155936,04_04,2,0.803501,1.0,1.0
29,experiment-04-04-sklearn-classification-logist...,run-20221104083407,04_04,3,0.803501,1.0,1.0
28,experiment-04-04-sklearn-classification-logist...,run-20221104090606,04_04,4,0.803501,1.0,1.0
27,experiment-04-04-sklearn-classification-logist...,run-20221107104554,04_04,5,0.803501,1.0,1.0
26,experiment-04-04-sklearn-classification-logist...,run-20221108143538,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
32,experiment-04-04c-sklearn-classification-logis...,run-20221220083637,04_04c,4,0.803501,1.0,1.0
31,experiment-04-04c-sklearn-classification-logis...,run-20230125100727,04_04c,5,0.803501,1.0,1.0
2,experiment-04-04d-sklearn-classification-logis...,run-20230131091335,04_04d,1,0.803501,1.0,1.0
1,experiment-04-04e-sklearn-classification-logis...,run-20230131095534,04_04e,1,0.803501,1.0,1.0


In [53]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
0,experiment-04-04f-sklearn-classification-logis...,run-20230201100605,04_04f,1,0.803501,1.0,1.0


In [54]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 1.0 within this experiment and 1.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [55]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1476393427452035072?project=mg-ce-demos


In [56]:
endpoint.display_name

'04'

In [57]:
endpoint.traffic_split

{'146257036726763520': 100}

In [58]:
deployed_models = endpoint.list_models()
deployed_models

[id: "146257036726763520"
 model: "projects/633472233130/locations/us-central1/models/model_04_04e"
 display_name: "04_04e"
 create_time {
   seconds: 1675178176
   nanos: 990976000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [59]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deply = True
    print('No models currently deployed.')

The current model is ranked better (1.0) than a currently deployed model (1.0).


### Deploy Model To Endpoint

In [60]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})') 

Deploying model with 100% of traffic...
Deploying Model projects/633472233130/locations/us-central1/models/model_04_04f to Endpoint : projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Deploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/1476393427452035072/operations/4363034556182298624
Endpoint model deployed. Resource name: projects/633472233130/locations/us-central1/endpoints/1476393427452035072


### Remove Deployed Models without Traffic

In [61]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Undeploying Endpoint model: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Undeploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/1476393427452035072/operations/4166565022438260736
Endpoint model undeployed. Resource name: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Undeploying 04_04e with version 1 because it has no traffic.
Model 04_04f with version 1 has traffic = 100


In [62]:
endpoint.traffic_split

{'2818017515664310272': 100}

In [63]:
endpoint.list_models()

[id: "2818017515664310272"
 model: "projects/633472233130/locations/us-central1/models/model_04_04f"
 display_name: "04_04f"
 create_time {
   seconds: 1675267367
   nanos: 21000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

---
## Prediction

See many more details on requesting predictions in the [04Tools - Prediction](./04Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [64]:
test_query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT.split('-'))]
y_test = test[VAR_TARGET]

instances = [X_test.to_dict(orient='split')['data'][0]]

### Get Predictions: Python Client

In [65]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[0.0], deployed_model_id='2818017515664310272', model_version_id='1', model_resource_name='projects/633472233130/locations/us-central1/models/model_04_04f', explanations=None)

In [66]:
prediction.predictions[0]

0.0

In [67]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [68]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": instances}))

In [69]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    0
  ],
  "deployedModelId": "2818017515664310272",
  "model": "projects/633472233130/locations/us-central1/models/model_04_04f",
  "modelDisplayName": "04_04f",
  "modelVersionId": "1"
}


### Get Predictions: gcloud (CLI)

In [70]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[0]


---
## Remove Resources
see notebook "99 - Cleanup"